### Install Detectron2 locally without internet (use kaggle dataset)

In [ ]:
!pip install ../input/detectron05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron05/whls 
!pip install ../input/detectron05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron05/whls 
!pip install ../input/detectron05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron05/whls 
!pip install ../input/detectron05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron05/whls 

### Import Detectron2 for Inference

In [3]:
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from PIL import Image
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fastcore.all import *
detectron2.__version__

### Helper Function

In [8]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def get_masks(fn, predictor):
    im = cv2.imread(str(fn))
    outputs = predictor(im)
    pred_masks = outputs['instances'].pred_masks.cpu().numpy()
    res = []
    used = np.zeros(im.shape[:2], dtype=int) 
    for mask in pred_masks:
        mask = mask * (1-used)
        used += mask
        res.append(rle_encode(mask))
    return res

### Load pre-trained model and setup detectron2 default predictor

In [12]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
cfg.MODEL.WEIGHTS = os.path.join('../input/sartoriusd2model', 'output', 'model_final.pth')  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.TEST.DETECTIONS_PER_IMAGE = 1000
predictor = DefaultPredictor(cfg)

### Prepare test images

In [16]:
data_dir = Path('../input/sartorius-cell-instance-segmentation')
ids, masks= [], []
test_names = (data_dir / 'test').ls()
test_names

#### Visualize prediction

In [20]:
encoded_masks = get_masks(test_names[2], predictor)
_, axs = plt.subplots(1,2, figsize=(40,30))
axs[1].imshow(cv2.imread(str(test_names[2])))
axs[1].axis("off")
mask = np.zeros((520, 704, 1))
for enc in encoded_masks:
    mask += rle_decode(enc, shape=(520, 704, 1))
    
mask = mask.clip(0, 1)
axs[0].imshow(mask)
axs[0].axis("off")

### Run model inference on test set and prepare submission file

In [17]:
for fn in test_names:
    encoded_masks = get_masks(fn, predictor)
    for enc in encoded_masks:
        ids.append(fn.stem)
        masks.append(enc)

In [18]:
pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()